In [60]:
from abc import ABC, abstractmethod

# Single responsibility principle

## Bad example (violates SRP)

In [61]:
class Report:
    def __init__(self, text):
        self.text = text

    def print_report(self): #handles printing 
        print(self.text)
    
    def save_to_file (self, filename): # handles saving
        with open(filename, "w") as f:
            f.write(self.text)
    """
    What is wrong?
    Report is 
    1- storing the data
    2- printing the report
    3- saving the report to a file

    3 responsibilities in 1 class
    if printing changes, the class must change
    if saving method chages (e.g., save to DB), the class must change
    violates SRP
    """

## Good Example (applies SRP)

In [62]:
class Report:
    def __init__ (self, text):
        self.text = text

class ReportPrinter:
    def print(self, report):
        print(report.text)

class ReportSaver:
    def save(self, report, filename):
        with open(filename, "w") as f:
            f.write (report.text)

"""
Now:
Report: only holds data
ReportPrinter: only prints
ReportSaver: only saves
each class has one reason to change
"""

'\nNow:\nReport: only holds data\nReportPrinter: only prints\nReportSaver: only saves\neach class has one reason to change\n'

In [63]:
class Order:
    def __init__(self,items):
        self.items = items

class OrderCalculator:
    def calculate_total(self, order):
        #self.item = Order.item
        return sum(order.items)
    
class PrintReceipt:
    def print_receipt(self,order,calculator ):
        i = 1
        for item in order.items:
            print(f"Item {i}: {item}")
            i = i + 1
        print(f"Total: {calculator.calculate_total(order)}")

class SaveOrder:
    def save_order(self,filename, order, calculator):
        with open(filename, "w") as f:
            f.write("Order Details:\n")
            for item in order.items:
                f.write(f"{item}\n")
            f.write(f"Total: {calculator.calculate_total(order)}")

In [64]:
order1 =  Order((12,5,21,5))
calculator = OrderCalculator()
calc_order1 = calculator.calculate_total(order1)
PrintReceipt1 = PrintReceipt()
PrintReceipt1.print_receipt(order1,calculator= OrderCalculator())

Item 1: 12
Item 2: 5
Item 3: 21
Item 4: 5
Total: 43


# Open closed principle

## Bad Example (Violates OCP)

In [65]:
class DiscountCalculator:
    def calculator (self, price, customer_type):
        if customer_type == "regular":
            return price
        elif customer_type == "vip":
            return price * 0.8 
        elif customer_type == "student":
            return price * 0.9
# every time you add a new customer type, you must modify this class --> BAD OCP

## Good example (Respects OCP)

### OCP using Abstraction with inheritance/polymorphism

In [66]:
class DiscountStrategy(ABC):
    @abstractmethod
    def apply(self, price):
        pass

class RegularDiscount (DiscountStrategy):
    def apply(self, price):
        return price
    
class VIPDsicount(DiscountStrategy):
    def apply(self, price):
        return price * 0.8
    
class StudentDiscount (DiscountStrategy):
    def apply(self, price):
        return price * 0.9
    
class DiscountCalculator:
    def calculator(self,price,discount_stratgy : DiscountStrategy):
        return discount_stratgy.apply(price)
    
calculator = DiscountCalculator()
print (calculator.calculator(200, VIPDsicount()))


160.0


In [67]:
class PaymentProcessor(ABC):
    @abstractmethod
    def processor_payment(self, amount):
        pass

class CreditCardPayment(PaymentProcessor):
    def processor_payment(self, amount):
        print (f"Processing credit card payment and the amount is {amount}")

class PayPalPayment(PaymentProcessor):
    def processor_payment(self, amount):
        print (f"Processing PayPal payment and the amount is {amount}")

class BankTransferPayment(PaymentProcessor):
    def processor_payment(self, amount):
        print (f"Processing bank transfer and the amount is {amount}")

class PaymentService:
    def __init__(self,Payment_method :PaymentProcessor ): # Dependency injection
        self.payment_method = Payment_method
    def pay(self,amount):
        self.payment_method.processor_payment(amount)
"""
1. Parameter name

payment_method → this is just the variable name.

2. Colon :

Means: “the expected type is…”

3. PaymentProcessor

This is the type hint:
We expect payment_method to be an instance of a class that inherits from(PaymentProcessor)
"""


'\n1. Parameter name\n\npayment_method → this is just the variable name.\n\n2. Colon :\n\nMeans: “the expected type is…”\n\n3. PaymentProcessor\n\nThis is the type hint:\nWe expect payment_method to be an instance of a class that inherits from(PaymentProcessor)\n'

In [68]:
payment_method = PayPalPayment()
service = PaymentService(payment_method)
service.pay(500)

Processing PayPal payment and the amount is 500


### OCP using composition 

In [69]:
class Order:
    def __init__(self, items, calculator):
        self.items = items
        self.calculator = calculator
    def total(self):
        return self.calculator.calculate_total(self) 
"""
here you can create new calculator classes without modifying Order and you can extend behavior by passing new strategies
this uses the strategy pattern, which is another OCP technique
"""

'\nhere you can create new calculator classes without modifying Order and you can extend behavior by passing new strategies\nthis uses the strategy pattern, which is another OCP technique\n'

# Liskov subistitution principle

## Bad Example (Violates LSP)

In [70]:
class Bird:
    def fly(self):
        return "Flying!"
    
class Penguin (Bird):
    def fly(self):
        raise Exception("Penguins can't fly!")
    
def make_bird_fly(bird: Bird):
    print(bird.fly())

# make_bird_fly(Penguin()) #this line will break the code



In [71]:
class Rectangle:
    def __init__(self, w, h):
        self.width = w
        self.height = h

    def set_width (self, w):
        self.width = w

    def set_height (self, h):
        self.height = h

class Square(Rectangle):
    def set_width(self, w):
        self.width = w
        self.height = w # force height = width 

    def set_height (self, h):
        self.height = h
        self.width = h # force width = hight 

In [72]:
# If the subclass requires more conditions, LSP is violated
class Payment:
    def pay(self, amount):
        print(f"Paying {amount}")

class SecurePayment (Payment):
    def pay(self, amount, password): # Requires more input which violates LSP
        if password != "1234":
            raise Exception("Invalid password")
        print(f"Securely paying {amount}")

In [73]:
# If the subclass does less conditions, LSP is violated
class Animal:
    def make_sound (self):
        print("Some generic sound")

class Snake (Animal):
    def make_sound(self):
        pass # does nothing     

## Good Example (Respects LSP)

In [74]:
class Bird:
    pass

class FlyingBird(Bird):
    def fly(self):
        return "Flying!"
    
class Penguin (Bird):
    def swim (self):
        return "Swimming!"

## Example

In [75]:


class File(ABC):
    @abstractmethod
    def open(self):
        pass


class Writable(ABC):
    @abstractmethod
    def write(self, data):
        pass


class ReadOnlyFile(File):
    def open(self):
        print("Opening read-only file...")


class ReadWriteFile(File, Writable):
    def open(self):
        print("Opening read-write file...")

    def write(self, data):
        print(f"Writing: {data}")


In [76]:
class Order (ABC):
    @abstractmethod 
    def get_total(self):
        pass

class FullPriceddOrder(Order):
    def __init__(self, items):
        self.items = items

    def get_total(self):
        return sum(self.items)
    
class DiscountOrder(Order):
    def __init__(self, items, discount_percent):
        self.items = items
        self.discount_percent = discount_percent

    def get_total(self):
        total = sum(self.items)
        
        return total - (total * self.discount_percent / 100)

# Interface segregation principle

## Bad Example (Violates ISP)

In [77]:
class Worker(ABC):
    @abstractmethod
    def work(self): pass

    @abstractmethod
    def eat(self): pass

    @abstractmethod
    def sleep (self): pass

In [78]:
# A RobotWorker
class RobotWorker (Worker):
    def work (self):
        print ("Robot working...")

    def eat (self): 
        raise Exception ("Robots dont eat!") # Violation 
    
    def sleep (self): 
        raise Exception ("Robots dont sleep!") # Violation 

## Good solution: split the interfaces (Respects LSP)

In [79]:
class Workable(ABC):
    @abstractmethod
    def work(self): pass

class Eatable (ABC):
    @abstractmethod
    def eat(self):
        pass

class Sleepable (ABC):
    @abstractmethod
    def sleep(self):
        pass

In [80]:
# Human worker:
class Human(Workable, Eatable, Sleepable):
    def work(self): print("Human working...")
    def eat(self): print("Human eating...")
    def sleep(self): print("Human sleeping...")

In [81]:
#Robot worker:
class Robot(Workable):
    def work(self): print("Robot working...")

## Composition Instead of Implementing a Big Interface

In [82]:
class WorkBehavior:
    def work(self):
        print ("working")

class EatBehavior:
    def eat(self):
        print ("eating")

class SleepBehavior:
    def Sleep(self):
        print ("Sleeping")                

In [83]:
class Robot:
    def __init__(self):
        self.work_behavior = WorkBehavior()

class Human:
    def __init__(self):
        self.work = WorkBehavior()
        self.eat = EatBehavior()
        self.sleep = SleepBehavior()

robot = Robot()
robot.work_behavior.work()

working


## excercise on ISP

In [84]:
class SmartDeviceSwitch(ABC):
    
    @abstractmethod       
    def turn_on(self): pass
        
    @abstractmethod
    def turn_off(self): pass

class SmartDeviceColorChangeable(ABC): 
    @abstractmethod
    def change_color(self, color): pass

class SmartDeviceTemperatureControl(ABC): 
    @abstractmethod
    def set_temperature(self, temp): pass

class RecordVIdeo(ABC):
    @abstractmethod
    def record_video(self): pass

class LiveStream(ABC):
    @abstractmethod
    def stream_live(self): pass 

class Music (ABC):
    @abstractmethod
    def play_music(self, song): pass
    
class VolumeControl(ABC):
    @abstractmethod
    def set_volume(self, level): pass

## Hard exercise 

In [85]:
class Notification (ABC):
    @abstractmethod
    def __init__(self, id, title, template_name, template_data, recipients, meta):
        self.id = id
        self.title = title    
        self.template_name = template_name    
        self.template_data = template_data 
        self.recipients = recipients
        self.meta = meta


class LowPriorityNotification(Notification):
    def __init__(self, id, title, template_name, template_data, recipients, meta):
        super().__init__(id, title, template_name, template_data, recipients, meta)
        self.priority = "Low"
    

class MidiumPriorityNotification(Notification):
    def __init__(self, id, title, template_name, template_data, recipients, meta):
        super().__init__(id, title, template_name, template_data, recipients, meta)
        self.priority = "Medium"

class HighPriorityNotification(Notification):
    def __init__(self, id, title, template_name, template_data, recipients, meta):
        super().__init__(id, title, template_name, template_data, recipients, meta)
        self.priority = "High"

In [86]:
n = LowPriorityNotification(id=1, title="x", template_name="t", template_data={}, recipients=[], meta={})

print(n.priority)

Low


In [87]:

class Channel(ABC):
    

    @abstractmethod
    def send(self,notification: Notification):
        pass 

class EmailChannel(Channel):
    

    def send(self,notification: Notification):
        email_address = notification.recipients 
        print (f"Sending email to {email_address}")
        print(f" subject is {notification.template_name}")
        print(f"message body: {notification.template_data} ")

class SMSChannel (Channel):

    def send(self,notification: Notification):
        phone_number = notification.recipients 
        print (f"Sending SMS to {phone_number}")
        print(f"message is: {notification.template_data} ")

class PushChannel (Channel):

    def send(self,notification: Notification):
        device_token = notification.recipients 
        print (f"device token is {device_token}")
        print(f" subject is {notification.template_name}")
        print(f"message body: {notification.template_data} ")




In [88]:
class DeliveryPolicy(ABC):
    @abstractmethod
    def get_channels(self, notification: Notification) -> list[Channel]:
        """Return the channels to use for this notification"""
        pass

# Example policies
class HighPriorityPolicy(DeliveryPolicy):
    def __init__(self, channels: list[Channel]):
        self.channels = channels  # Injected channels, only abstraction

    def get_channels(self, notification: Notification):
        # High priority uses all available channels
        return self.channels

class LowPriorityPolicy(DeliveryPolicy):
    def __init__(self, channels: list[Channel]):
        self.channels = channels

    def get_channels(self, notification: Notification):
        # Low priority might use only one or two channels
        return [ch for ch in self.channels if isinstance(ch, EmailChannel)]

In [89]:
class NotificationService:
    def __init__(self, delivery_policy: DeliveryPolicy):
        self.delivery_policy = delivery_policy
        
        

    def send(self, notification: Notification):
        channels_to_use = self.delivery_policy.get_channels(notification)
        for channel in channels_to_use:
            channel.send(notification)
        
       

In [90]:
notification1 = MidiumPriorityNotification(12, "Hello", "greatings", {1:"hello friend", 2:"glad to meet you" }, (11,2,4,5), {1:"EncodingWarning"}  )

service = NotificationService(
   HighPriorityPolicy( channels=[EmailChannel(), SMSChannel(), PushChannel()])
   )
service.send(notification1)


Sending email to (11, 2, 4, 5)
 subject is greatings
message body: {1: 'hello friend', 2: 'glad to meet you'} 
Sending SMS to (11, 2, 4, 5)
message is: {1: 'hello friend', 2: 'glad to meet you'} 
device token is (11, 2, 4, 5)
 subject is greatings
message body: {1: 'hello friend', 2: 'glad to meet you'} 
